### If you want to create new ROI file on your own then DELETE the already existing file in the shared file folder
### In this folder ROI file named as 'ROI_Cam4_0&1'

### New file will be created automatically when you run the code to define new ROI

In [1]:
import cv2
import numpy as np
import cvzone
import pickle

#Instructions STEP 2:
#Give path to your video file inside double quotes [""]
cap = cv2.VideoCapture("Edge_Crossing_1_Cam4_0&1.avi")
drawing = False
area_list = []
roi_points = []

try:
    #Instructions STEP 3.1:
    with open("ROI_Cam4_0&1", "rb") as f:
        data = pickle.load(f)
        polylines, area_list = data['polylines'], data['area_list']
except:
    polylines = []

points = []
current_name = ""

def draw(event, x, y, flags, param):
    global points, drawing, roi_points
    if roi_type == "hover":
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            points = [(x, y)]
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                points.append((x, y))
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            current_name = input("areaname:- ")
            if current_name:
                area_list.append(current_name)
                polylines.append(np.array(points, np.int32))
    elif roi_type == "points":
        if event == cv2.EVENT_LBUTTONDOWN:
            roi_points.append((x, y))
            if len(roi_points) == 4:
                roi_polygon = np.array(roi_points, np.int32).reshape((-1, 1, 2))
                polylines.append(roi_polygon)
                area_list.append("ROI")
                roi_points = []

while True:
    roi_type = input("Enter ROI type ('points' or 'hover'): ").lower()
    if roi_type not in ["hover", "points"]:
        print("Invalid input. Please enter 'hover' or 'points'.")
        continue
    break

while True:
    ret, frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue
    frame = cv2.resize(frame, (640, 640))

    for i, polyline in enumerate(polylines):
        cv2.polylines(frame, [polyline], isClosed=True, color=(0, 0, 255), thickness=2)
        if len(polyline) > 1 and i < len(area_list):
            if isinstance(polyline[0][0], np.int32):
                text_position = (polyline[0][0], polyline[0][1])
            else:
                text_position = tuple(polyline[0][0])
            cvzone.putTextRect(frame, f'{area_list[i]}', text_position, 1, 1)

    cv2.imshow('FRAME', frame)
    cv2.setMouseCallback('FRAME', draw)

    key = cv2.waitKey(0) & 0xFF
    if key == ord('s'):
        #Instructions STEP 3.2:
        with open("ROI_Cam4_0&1", "wb") as f:
            data = {'polylines': polylines, 'area_list': area_list}
            pickle.dump(data, f)
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()